### **One Alien, One Crew Member**

In [1]:
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt
from math import exp
import heapq
from pprint import pprint

In [2]:
def seed_ship(D):    # seed the ship with a single open cell with ship size = D
    M = np.zeros((D,D),dtype=bool)
    rand_seed = (np.random.randint(0,D),np.random.randint(0,D))
    M[rand_seed] = True
    non0=[]
    non0.append((rand_seed))
    zero = []
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            if M[i][j] == False:
                zero.append((i,j))
    # print(non0)
    return M, non0, zero

def neighbors(M,nz):   # utility function to find open and closed neighbors of a cell
    moves = ((-1,0), (0,-1), (1,0), (0,1))
    possible = [(nz[0]+i[0],nz[1]+i[1]) for i in moves]
    blocked_cells = []
    open_cells = []
    for i in possible:
        if i[0]>=0 and i[0]<M.shape[0] and i[1]>=0 and i[1]<M.shape[0]:
            if M[i] == False:
                blocked_cells.append(i)
            else:
                open_cells.append(i)

    return blocked_cells,open_cells

# Display the current state of the ship
def display_current(M, bot_position=[(0,0)], crew=[(0,0)], alien_positions=[(0,0)], path=[]):
    crewcount=1
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            if (M[i][j] == False):
                print("⬛", end="")
            else:
                if (i,j) == bot_position[0]:
                    print("🤖", end="")
                elif (i,j) in crew:
                    if crewcount==1:
                        print("😨", end="")
                    if crewcount==2:
                        print("😥", end="")
                    crewcount+=1
                elif (i,j) in alien_positions:
                    print("💀",end="")
                elif (i,j) in path:
                    print("*️⃣",end="")
                else:
                    print("🟩",end="")
        print()

def gen_ship(M,open_cells,closed_cells):
    def closed_candidates():        # Figure out all possible closed cells that can be opened.
        candidates = []             # For all closed cells if they have exactly 1 open neighbor,
        for j in closed_cells:      # add those to a list from which later we choose one randomly to open
            _,on = neighbors(M,j)
            if len(on) == 1:
                candidates.append(j)
        return candidates
    candidates = closed_candidates()

    while len(candidates)>0:                   # Until the candidate list is exhausted, choose one random
        open_this = random.choice(candidates)  # candidate and open it, then calculate the list again.
        M[open_this] = True
        closed_cells.remove(open_this)
        open_cells.append(open_this)
        candidates = closed_candidates()

    dead_end_candidates = []                 # Figure out the list of dead ends.
    for i in open_cells:                     # Choose 1 neighbor of the DE at random and add it to the list
        cn,on = neighbors(M,i)
        if len(on) == 1 and len(cn)>0:
            open_this = random.choice(cn)
            dead_end_candidates.append(open_this)

    dead_end_candidates = set(dead_end_candidates) # remove duplicates
    dead_end_candidates = list(dead_end_candidates)
    random.shuffle(dead_end_candidates)             # For a random half of the dead ends,
    for j in range(len(dead_end_candidates)//2):    # open one closed neighbor each
        open_this = dead_end_candidates[j]
        if open_this not in closed_cells:
            print(open_this)
        M[open_this] = True
        closed_cells.remove(open_this)
        open_cells.append(open_this)

# Generate an adjacency list for the grid
def adjacency_list(M):
    ADJ = {}
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            if M[i,j] == True:
                _, oc = neighbors(M,(i,j))
                ADJ[(i,j)] = oc
    return ADJ

def weight_adj_list(M,belief):
    ADJ = {}
    for i in range(M.shape[0]):
        for j in range(M.shape[1]):
            if M[i,j] == True:
                _, oc = neighbors(M,(i,j))
                for k in oc:
                    if (i,j) not in ADJ.keys():
                        ADJ[(i,j)] = [(k,belief[0,k[0],k[1]])]    #i,j --> k[0], k[1] should have the next node's weights
                    else:
                        ADJ[(i,j)].append((k,belief[0,k[0],k[1]]))
    return ADJ

def shortest_path(ADJ,bot_position,captain_position):   #Uses BFS to find the shortest path from the bot to the captain
    queue = deque()
    visited = []
    epic_tuple = bot_position  #a workaround due to a stupid error
    queue.append(epic_tuple)      #start with the position of the bot
    prev = {}
    path_exists = False
    while queue:
        now = queue.popleft()
        for child in ADJ[now]:
            if child not in visited:
                visited.append(now)
                queue.append(child)
                prev[child] = now
            if child == captain_position:
                path_exists = True
                break

    now = captain_position
    path = [captain_position]
    if not path_exists:
        return [bot_position,bot_position]
    if bot_position == captain_position:
        return path
    
    while prev[now] != bot_position:
        now = prev[now]
        path.append(now)

    path.append(bot_position)
    list.reverse(path)
    return path

def precompute_distance(open_cells, crew):
    dist = {}
    for i in open_cells:
        if len(crew) == 1:
            dist[i] = (abs(i[0]-crew[0][0])+abs(i[1]-crew[0][1]),None)
        if len(crew) == 2:
            dist[i] = (abs(i[0]-crew[0][0])+abs(i[1]-crew[0][1]),abs(i[0]-crew[1][0])+abs(i[1]-crew[1][1]))
    return dist

def set_players(open_cells, num_aliens=1, num_crew=1, k=1):
    bot_position, crew_positions, alien_positions = [(0, 0)], [(0, 0)], [(0, 0)]

    if num_aliens >= len(open_cells):
        print("Too many aliens. The bot and captain will surely die")
        return bot_position, crew_positions, alien_positions

    bot_idx = random.choice(
        [i for i in range(len(open_cells))]
    )

    bot_position[0] = open_cells[bot_idx]
    crew_idx = [random.choice([i for i in range(len(open_cells)) if i not in [bot_idx]]) for _ in range(num_crew)]
    crew_positions = [open_cells[i] for i in crew_idx]

    alien_choice = []
    for i in range(num_aliens):
        for j in open_cells:
            if j not in crew_positions and j != bot_position[0]:
                if abs(j[0] - bot_position[0][0]) > k and abs(j[1] - bot_position[0][1]) > k:
                    alien_choice.append(j)

    alien_positions = [random.choice([i for i in alien_choice]) for _ in range(num_aliens)]

    return bot_position, crew_positions, alien_positions

def move_aliens(M, alien_positions, bot_position):
    random.shuffle(alien_positions)     #randomize alien actions to simulate their free will

    for i in range(len(alien_positions)):
        _,oc = neighbors(M, alien_positions[i])
        for j in oc:                    # for j in open neighbors
            if j in alien_positions:
                oc.remove(j)            # stop aliens from bumping into each other
        if len(oc)>=1:                  # if false, the alien stays in its place
            next_posi = random.choice(oc)
            alien_positions[i] = next_posi
    if bot_position[0] in alien_positions:
        return "aliens killed bot"
    else:
        return "nothing happened"

def a_star(start, goal, ADJ):
    # Priority queue: stores tuples of (estimated_cost, current_path_cost, node, path)
    priority_queue = [(0 + abs(start[0] - goal[0]) + abs(start[1] - goal[1]), 0, start, [start])]
    # Visited nodes to keep track of the minimum cost to reach a node
    visited = {start: 0}
    
    while priority_queue:
        # Get the current node with the lowest estimated cost
        estimated_cost, current_cost, current_node, path = heapq.heappop(priority_queue)
        
        # If the goal is reached, return the path
        if current_node == goal:
            return path
        
        # Explore neighbors
        for neighbor, cost in ADJ[current_node]:
            new_cost = current_cost + cost
            # If this path is better than any previous one, record it
            if neighbor not in visited or new_cost < visited[neighbor]:
                visited[neighbor] = new_cost
                new_path = path + [neighbor]
                estimated_cost = new_cost + abs(neighbor[0] - goal[0]) + abs(neighbor[1] - goal[1])
                heapq.heappush(priority_queue, (estimated_cost, new_cost, neighbor, new_path))
    
    return None  # If there's no path

def bot1_move(M, bot_position, alien_positions, captain_position, belief, ADJ):
    if bot_position[0] in captain_position:
        return "bot saved captain"

    belief_score = belief[1] - belief[0]

    goal = np.unravel_index(np.argmax(belief_score), belief_score.shape)
        
    path = a_star(bot_position[0], goal, ADJ)
    if len(path) > 1:
        bot_position[0] = path[1]

    if bot_position[0] in alien_positions:
        return "aliens killed bot"
    elif bot_position[0]==captain_position:
        return "bot saved captain"
    else:
        return "nothing happened"

def bot2_move(M, bot_position, alien_positions, captain_position, belief, ADJ):
    if bot_position[0] in captain_position:
        return "bot saved captain"

    belief_score = 0.5*belief[1] - 2*belief[0]

    goal = np.unravel_index(np.argmax(belief_score), belief_score.shape)
    # random.choice(np.unravel_index(np.argwhere(belief_score == np.amax(belief_score)), belief_score.shape))
    # print(type(goal),goal)

    path = a_star(bot_position[0], goal, ADJ)
    if len(path) > 1:
        bot_position[0] = path[1]

    if bot_position[0] in alien_positions:
        return "aliens killed bot"
    elif bot_position[0]==captain_position:
        return "bot saved captain"
    else:
        return "nothing happened"

def alien_beep(bot_position, alien_positions, k):
    for j in alien_positions:
        if abs(j[0] - bot_position[0][0]) <= k and abs(j[1] - bot_position[0][1]) <= k:
            return True
    return False

def crew_beep(bot_position, crew, dist, alpha=0.1):
    if len(crew) == 2:
        d = dist[bot_position[0]]
        p = exp(-alpha*(d[0]-1)) + exp(-alpha*(d[1]-1))
        p = p / 2
        s = random.uniform(0,1)
        if s < p:
            return True
        else:
            return False
    else:
        d = dist[bot_position[0]]
        p = exp(-alpha*(d[0]))
        s = random.uniform(0,1)
        if s < p:
            return True
        else:
            return False       

In [3]:
def init_belief(M, open_cells, k, bot_position):
    belief = np.zeros((2, M.shape[0], M.shape[1]))
    initial_safe_box = []
    for cell in open_cells:
        if abs(cell[0] - bot_position[0][0]) <= k and abs(cell[1] - bot_position[0][1]) <=k:
            initial_safe_box.append(cell)  
        belief[1][cell[0], cell[1]] = 1 / len(open_cells)

    for cell in open_cells:
        if cell not in initial_safe_box:
            belief[0][cell[0], cell[1]] = 1 / (len(open_cells) - len(initial_safe_box))
        
    return belief

def update_probs_sense(
    belief,
    open_cells,
    bot_position,
    crew_beeped,
    alien_beeped,
    alpha=0.5,
    k = 2
):
    belief[0, bot_position[0][0], bot_position[0][1]] = 0  # alien can't be in bot's position
    belief[1, bot_position[0][0], bot_position[0][1]] = 0  # crew hasn't been rescued yet 
                                                           # so no chance of it being in bot's position
    alien_detection_region = []
    for cell in open_cells:
        if abs(cell[0] - bot_position[0][0]) <= k and abs(cell[1] - bot_position[0][1]) <=k:
            alien_detection_region.append(cell)

    if crew_beeped:
        p_beep_in_j = 0
        for cell in open_cells:
            d_k = abs(cell[0]-bot_position[0][0]) + abs(cell[1] - bot_position[0][1])
            p_beep_in_j += belief[1][cell[0], cell[1]]*exp(-alpha * (d_k - 1))

        for cell in open_cells:
            d = abs(cell[0]-bot_position[0][0]) + abs(cell[1] - bot_position[0][1])
            belief[1][cell[0], cell[1]] = (belief[1][cell[0], cell[1]]*exp(-alpha * (d-1))) / p_beep_in_j

    elif not crew_beeped:
        p_no_beep_in_j = 0
        for cell in open_cells:
            d_k = abs(cell[0]-bot_position[0][0]) + abs(cell[1] - bot_position[0][1])
            p_no_beep_in_j += belief[1][cell[0], cell[1]]*(1 - exp(-alpha * (d_k - 1)))
            
        for cell in open_cells:
            d = abs(cell[0]-bot_position[0][0]) + abs(cell[1] - bot_position[0][1])
            belief[1][cell[0], cell[1]] = (belief[1][cell[0], cell[1]]*(1 - exp(-alpha * (d-1)))) / p_no_beep_in_j

    if alien_beeped:
        p_alien_in_sensor = 0
        for i in alien_detection_region:
            p_alien_in_sensor += belief[0, i[0], i[1]]

        if p_alien_in_sensor == 0:
            for i in open_cells:
                if i in alien_detection_region:
                    belief[0, i[0], i[1]] = 1 / len(open_cells)
                else:
                    belief[0, i[0], i[1]] = 0
            return belief

        for i in open_cells:
            if i not in alien_detection_region:
                belief[0, i[0], i[1]] = 0
            if i in alien_detection_region:
                belief[0, i[0], i[1]] = belief[0, i[0], i[1]] / p_alien_in_sensor

    elif not alien_beeped:
        p_alien_not_in_sensor = 0
        for i in alien_detection_region:
            belief[0, i[0], i[1]] = 0
        belief[0, bot_position[0][0], bot_position[0][1]] = 0

        for i in open_cells:
            if i not in alien_detection_region:
                p_alien_not_in_sensor += belief[0, i[0], i[1]]
        for i in open_cells:
            if i not in alien_detection_region:
                belief[0, i[0], i[1]] = belief[0, i[0], i[1]] / (1 - p_alien_not_in_sensor)


    return belief

def update_probs_after_bot_move(belief, open_cells,bot_position):
    
    beliefc = belief.copy() # make a copy of the prior

    # update probs for the crew and aliens
    for cell in open_cells:
            beliefc[1, cell[0], cell[1]] = belief[1, cell[0], cell[1]] \
                / (1 - belief[1, bot_position[0][0], bot_position[0][1]])
            if not belief[0, cell[0], cell[1]] == 1:
                beliefc[0, cell[0], cell[1]] = belief[0, cell[0], cell[1]] \
                    / (1 - belief[0, bot_position[0][0], bot_position[0][1]])
                
    beliefc[0, bot_position[0][0], bot_position[0][1]] = 0 # the alien isn't there
    beliefc[1, bot_position[0][0], bot_position[0][1]] = 0 # the crew isn't there either
    belief = beliefc

def update_probs_after_alien_move(M, belief, open_cells, bot_position):
    
    beliefc = belief[0].copy() # make a copy of the prior

    for cell in open_cells:
        _,oc = neighbors(M, cell)
        prob_cell = 0
        for c in oc:
            prob_cell += (belief[0, c[0], c[1]] * 1/len(oc))
        beliefc[cell[0], cell[1]] = prob_cell/(1-belief[0,c[0],c[1]])

    belief[0] = beliefc

In [4]:
# game loop for bots
def game_loop(bot):
    np.set_printoptions(precision=2)
    k_values = [i for i in range(2, 9)]
    # k_values = [2]
    alpha_values = np.arange(0.05, 0.5, 0.05)
    # alpha_values = [0.3]
    iterations = 1000
    D = 35

    success_states = []   # store (1, alpha, k, iterations) if saved
    loss_states = []      # store (1, alpha, k, iterations) if killed
    stalemate_states = [] # store (1, alpha, k, iterations) if nothing happened
    num_ships = 5
    for k in k_values:
        # display_current(M,bot_position,crew_positions,alien_positions)
        for alpha in alpha_values:
                for ship_number in range(num_ships):
                    M, open_cells, closed_cells = seed_ship(D)
                    gen_ship(M, open_cells, closed_cells)
                    bot_position, crew_positions, alien_positions = set_players(
                    open_cells, num_aliens=1, num_crew=1, k=k
                    )
                    dist = precompute_distance(open_cells, crew_positions)

                    belief = init_belief(M, open_cells, k, bot_position)
                    weight_ADJ = weight_adj_list(M, belief)
                    state = "nothing happened"
                    for it in range(iterations):
                        crew_beeped = crew_beep(bot_position, crew_positions, dist, alpha=alpha)
                        alien_beeped = alien_beep(bot_position, alien_positions, k)
                        update_probs_sense(
                            belief,
                            open_cells,
                            bot_position,
                            crew_beeped,
                            alien_beeped,
                            alpha=alpha,
                            k=k,
                        )

                        state = bot(
                            M, bot_position, alien_positions, crew_positions, belief, weight_ADJ
                        )

                        if state.lower() == "aliens killed bot":
                            success_states.append((False, alpha, k, it, ship_number + 1))
                            loss_states.append((True, alpha, k, it, ship_number + 1))
                            stalemate_states.append((False, alpha, k, it, ship_number + 1))
                            break
                        elif state.lower() == "bot saved captain":
                            success_states.append((True, alpha, k, it, ship_number + 1))
                            loss_states.append((False, alpha, k, it, ship_number + 1))
                            stalemate_states.append((False, alpha, k, it, ship_number + 1))
                            break
                        else:
                            update_probs_after_bot_move(
                                belief,
                                open_cells,
                                bot_position,
                            )
                        state = move_aliens(M, alien_positions, bot_position)
                        if state.lower() == "aliens killed bot":
                            success_states.append((False, alpha, k, it, ship_number + 1))
                            loss_states.append((True, alpha, k, it, ship_number + 1))
                            stalemate_states.append((False, alpha, k, it, ship_number + 1))
                            break
                        else:
                            update_probs_after_alien_move(
                                M,
                                belief,
                                open_cells,
                                bot_position,
                            )

                    if state == "nothing happened":
                        stalemate_states.append((True, alpha, k, it, ship_number + 1))
                        success_states.append((False, alpha, k, it, ship_number + 1))
                        loss_states.append((False, alpha, k, it, ship_number + 1))

    data = [success_states, loss_states, stalemate_states]
    return data

In [ ]:
bot1_data = game_loop(bot=bot1_move)
bot2_data = game_loop(bot=bot2_move)

In [6]:
success_states_bot1 = np.array(bot1_data[0])
loss_states_bot1 = np.array(bot1_data[1])
stalemate_states_bot1 = np.array(bot1_data[2])

success_states_bot2 = np.array(bot2_data[0])
loss_states_bot2 = np.array(bot2_data[1])
stalemate_states_bot2 = np.array(bot2_data[2])

In [ ]:
avg_success_rate = np.sum(success_states_bot1[:,0]) / success_states_bot1.shape[0]
avg_loss_rate = np.sum(loss_states_bot1[:,0]) / loss_states_bot1.shape[0]
avg_stalemate_rate = np.sum(stalemate_states_bot1[:,0]) / stalemate_states_bot1.shape[0]
print(avg_success_rate, avg_loss_rate, avg_stalemate_rate)

In [ ]:
avg_success_rate = np.sum(success_states_bot2[:,0]) / success_states_bot2.shape[0]
avg_loss_rate = np.sum(loss_states_bot2[:,0]) / loss_states_bot2.shape[0]
avg_stalemate_rate = np.sum(stalemate_states_bot2[:,0]) / stalemate_states_bot2.shape[0]
print(avg_success_rate, avg_loss_rate, avg_stalemate_rate)